# **Введение в анализ данных**
# **НИУ ВШЭ, 2019-2020 учебный год**
**Домашнее задание №3**

Задание выполнил: Соснин Артём

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [ ]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

--2020-06-17 11:48:48--  https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/tg55q9mrziroyrs/train_subset.csv [following]
--2020-06-17 11:48:48--  https://www.dropbox.com/s/raw/tg55q9mrziroyrs/train_subset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2d29d6e092d5be72b33f569911.dl.dropboxusercontent.com/cd/0/inline/A51kF69ImZCRaryqEQl6-RwpDWEtZJr6E_QmXDm7xRK14LmSlXKIr9zPJgeDnm0bc3LKXMgVqKE98tJ9E_oJWrWLL69krXPLl3_ozg0aJM5bhueuImKm7XxoogTdQyzvzKM/file# [following]
--2020-06-17 11:48:48--  https://uc2d29d6e092d5be72b33f569911.dl.dropboxusercontent.com/cd/0/inline/A51kF69ImZCRaryqEQl6-RwpDWEtZJr6E_QmXDm7xRK14LmSlXKIr9zPJgeDnm0bc3LKXMgVqKE98tJ9E_oJWrWLL69krXPLl3_

**Данные**  
Мы имеем дело с данными с торговой платформы Avito. Для каждого товара представлены следующие параметры:  

*   title
*   description
*   Category_name
*   Category  
Имеется информация об объектах 50 классов. Задача: по новым объектам (title, description) предсказать Category. (Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [ ]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [ ]:
data.shape

(30000, 4)

In [ ]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test. Никакие данные из test для обучения использовать нельзя!

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

**Токенизация (1 балл)**  
Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами. Можно использовать разные алгоритмы токенизации. Давайте пока остановимся на простом WordPunctTokenizer.

In [ ]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я3, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я3, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я3 , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


Задание: Токенизируйте title и description в train и test

In [ ]:
X_train=np.asarray([np.asarray([preprocess(s) for s in X_train[i]]) for i in range(X_train.shape[0])])
X_test=np.asarray([np.asarray([preprocess(s) for s in X_test[i]]) for i in range(X_test.shape[0])])

In [ ]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [ ]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

**BOW (1.5 балла)**  
Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

Составить словарь самых часто встречающихся слов в train data  
Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается  
В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

Задание: Найдите k самых частых слов, отсортируйте их по убыванию частотности (k=10000)

Сначала создадим массив всех слов allwords, которые встречаются в X_train 

In [ ]:
allwords=np.concatenate(np.concatenate(np.asarray([[ sr.split() for sr in X_train[i]] for i in range(X_train.shape[0])])))

Теперь с помощью Counter, создадим словарь bow_vocabulary, где для каждого слова из allwords указано колочество раз, которое оно встречается в allwords.

In [ ]:
from collections import Counter
bow_vocabulary=Counter(allwords)

Отсортируем bow_vocabulary по частоте встречаемости слов. bow - это отсортированный список 10000 самых популярных слов.

In [ ]:
bow=sorted(bow_vocabulary, key=bow_vocabulary.get,reverse=True)[:10000]

In [ ]:
# только здесь то, что подразумевалось под  bow_vocabulary у меня называется bow
assert sorted(bow)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения','габариты',
'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 
'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску','постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 
'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [ ]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    words=text.split()
    return np.asarray([words.count(word) for word in bow])

In [ ]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [ ]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    
    return np.asarray([text_to_bow(item[1]) for item in items])
   


In [ ]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

In [ ]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

**Логистическая регрессия и SVC (1 балл)**  
Теперь описание каждого товара представлено, как точка в многомерном пространстве. Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово. Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.

Так как матрицы X_train, X_test содержать много нулей, дальше я буду работать с разреженными матрицами, чтобы уменьшить объем используемой памяти.

In [ ]:
from scipy.sparse import csr_matrix
X_test_bow_csr = csr_matrix(X_test_bow)
X_train_bow_csr = csr_matrix(X_train_bow)

del X_test_bow
del X_train_bow

In [ ]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_csr, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_csr), y_test))

assert accuracy_score(bow_model.predict(X_test_bow_csr), y_test) > 0.7

0.7011111111111111


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
from sklearn.svm import LinearSVC

bow_model = LinearSVC(max_iter=70).fit(X_train_bow_csr, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_csr), y_test))

assert accuracy_score(bow_model.predict(X_test_bow_csr), y_test) > 0.68

0.6828888888888889


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


**Модификация признаков (0.5 балла)**  
Добавьте title товара в bow с произвольным весом, как изменится качество?

In [ ]:
def items_to_bow_title(items: np.array) -> np.array:
    """ аналогичная функция items_to_bow, только теперь возвращает вектор для title товара"""
    
    return np.asarray([text_to_bow(item[0]) for item in items])

Сложим для каждого товара вектора его title и description, вектор title возьмем с весом 2, так получается лучшее качество. 

In [111]:
X_train_bow_cdt=csr_matrix(X_train_bow_csr+2*items_to_bow_title(X_train))
X_test_bow_cdt=csr_matrix(X_test_bow_csr+2*items_to_bow_title(X_test))

In [112]:
model = LogisticRegression(max_iter=100).fit(X_train_bow_cdt, y_train)
print('Качество логистической регресии: ',accuracy_score(model.predict(X_test_bow_cdt), y_test))

Качество логистической регресии:  0.7943333333333333


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [113]:
model = LinearSVC(max_iter=70).fit(X_train_bow_cdt, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_bow_cdt), y_test))

Качество SVC:  0.7625555555555555


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Как видно, качество улучшилось примерно на 0,09 для логистической регрессии и на 0,08 для SVC

Нормализуйте данные (sklearn.preprocessing.normalize) перед обучением. Что станет с качеством и почему?

In [114]:
from sklearn.preprocessing import normalize
X_train_n=csr_matrix(normalize(X_train_bow_cdt))
X_test_n=csr_matrix(normalize(X_test_bow_cdt))

In [115]:
model = LogisticRegression(max_iter=100).fit(X_train_n, y_train)
print('Качество логистической регресии: ',accuracy_score(model.predict(X_test_n), y_test))

Качество логистической регресии:  0.7041111111111111


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [116]:
model = LinearSVC(max_iter=70).fit(X_train_n, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_n), y_test))

Качество SVC:  0.8072222222222222


Что ж на SVC качестов увеличилось, а на логисточеской регрессии упало. При нормализации значения всех признаков переводятся в диапозон от 0 до 1. В нашем случаи это снижает качество логистической регресси.



**mystem (0.5) балла**  
Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [117]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-06-17 11:58:19--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.241, 5.45.205.245, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-06-17 11:58:19--  http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)... 5.45.220.16, 2a02:6b8:0:2002::17
Connecting to cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)|5.45.220.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

In [118]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-fvnhj71f
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-fvnhj71f
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9921 sha256=a24b2e0a1fbbfaba49e4eea03672210b67fce032b26a6ead4d853c7003bdd10c
  Stored in directory: /tmp/pip-ephem-wheel-cache-p0qjq67h/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


In [119]:
from pymystem3 import Mystem
m=Mystem()

In [120]:
def lemmat (text: str) -> str:
    return ''.join(m.lemmatize(text)[:-1])

Обработает строки выборки с помощью лемматизатора Mystem (ниже приведен пример)

In [121]:
lemmat('Сапоги 46 размер новые')

'сапог 46 размер новый'

In [122]:
X_train_lem=np.asarray([np.asarray([lemmat(s) for s in X_train[i]]) for i in range(X_train.shape[0])])
X_test_lem=np.asarray([np.asarray([lemmat(s) for s in X_test[i]]) for i in range(X_test.shape[0])])

Для того чтобы было работать с CountVectorizer, для каждого товара сложим строки его title и description

In [123]:
X_train_cv_lem=[item[0]+' '+item[1] for item in X_train_lem]
X_test_cv_lem=[item[0]+' '+item[1] for item in X_test_lem]

In [124]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train_cv_lem=cv.fit_transform(X_train_cv_lem)
X_test_cv_lem=cv.transform(X_test_cv_lem)


In [125]:
model = LogisticRegression(max_iter=100).fit(X_train_cv_lem, y_train)
print('Качество логистической регресии: ', accuracy_score(model.predict(X_test_cv_lem), y_test))

Качество логистической регресии:  0.8016666666666666


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [126]:
model = LinearSVC(max_iter=70).fit(X_train_cv_lem, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_cv_lem), y_test))

Качество SVC:  0.7891111111111111


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Теперь нормализуем данные.

In [127]:
X_train_lem_n=csr_matrix(normalize(X_train_cv_lem))
X_test_lem_n=csr_matrix(normalize(X_test_cv_lem))

In [128]:
model = LogisticRegression(max_iter=100).fit(X_train_lem_n, y_train)
print('Качество логистической регресии: ',accuracy_score(model.predict(X_test_lem_n), y_test))

Качество логистической регресии:  0.7664444444444445


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [129]:
model = LinearSVC(max_iter=70).fit(X_train_lem_n, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_lem_n), y_test))

Качество SVC:  0.8314444444444444


В итоге, с помощью лемматизатора mystem удалось повысить качество как и логистической регрессии, так и SVC, после  нормализации качество логистической регрессии опять упало, но уже не так критично, а на SVC поднялось до рекордных пока 0.8314444444444444. 

**TF-IDF (1.5 балла)**  
Не все слова полезны одинаково, давайте попробуем взвесить их, чтобы отобрать более полезные.

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).

IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [130]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
# в тексте скольких товаров встретилось это слово

text=[set(np.concatenate([sr.split() for sr in X_train[i]])) for i in range(X_train.shape[0])]
count_arr=dict()
for word in bow:
  count_arr[word]=sum([1 for s in text if word in s])

count_arr - это словарь, где для 10000 самых частвовстречающихся слов (у меня  это bow) указано в скольких описаниях товаров они встретились ( описание это title+description)

In [131]:
from math import log

In [132]:
def text_to_tfidf(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    words=text.split()
    return np.asarray([words.count(word)/len(words)*log(21000/count_arr[word]) for word in bow])

In [133]:
def items_to_tfidf(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его tf-idf"""
    
    return np.asarray([text_to_tfidf(item[0]+' '+item[1]) for item in items])

tf-idf для товара считается от сложения его строк title и description

In [134]:
X_train_tfidf = csr_matrix(items_to_tfidf(X_train))
X_test_tfidf = csr_matrix(items_to_tfidf(X_test))

In [135]:
# Нормализуйте данные
from sklearn.preprocessing import normalize
X_train_tf_n=csr_matrix(normalize(X_train_tfidf))
X_test_tf_n=csr_matrix(normalize(X_test_tfidf))

**Модели на TF-IDF признаках (1 балл)**  
Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [136]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=100).fit(X_train_tf_n, y_train)
print('Качество логистической регресии: ',accuracy_score(model.predict(X_test_tf_n), y_test))

Качество логистической регресии:  0.7653333333333333


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [137]:
from sklearn.svm import LinearSVC

model = LinearSVC(max_iter=70).fit(X_train_tf_n, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_tf_n), y_test))

Качество SVC:  0.8067777777777778


Что ж если сравнивать с методом BOW после нормализации, то качество на логистической регрессии удалось повысить на 0,06 , а на SVC оно почти не изменилось. В общем, качество достаточно высокое, но проирывает полученному с помощью лемматизатора mystem.

**Hashing Vectorizer (0.5 балла)**  
Попробуйте использовать sklearn.feature_extraction.text.HashingVectorizer для векторизации текстов. Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [138]:
from sklearn.feature_extraction.text import HashingVectorizer

Опять сложим title и description каждого товара для того чтобы, воспользоваться HashingVectorizer

In [139]:
X_train_vec=np.asarray([X_train[i][0]+' '+X_train[i][1] for i in range(X_train.shape[0])])
X_test_vec=np.asarray([X_test[i][0]+' '+X_test[i][1] for i in range(X_test.shape[0])])

In [140]:
vectorizer = HashingVectorizer(n_features=100000)
X_train_hash=vectorizer.fit_transform(X_train_vec)
X_test_hash=vectorizer.transform(X_test_vec)

In [141]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_hash, y_train)
print('Качество логистической регресии: ',accuracy_score(bow_model.predict(X_test_hash), y_test))

Качество логистической регресии:  0.7371111111111112


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [142]:
from sklearn.svm import LinearSVC

model = LinearSVC(max_iter=70).fit(X_train_hash, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_hash), y_test))

Качество SVC:  0.8247777777777778


**Word Vectors (1 балл)**  
Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [143]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-06-17 12:13:35--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-06-17 12:13:35--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce509eadd95d70f30bc44c98d08.dl.dropboxusercontent.com/cd/0/inline/A50F8T3uwdh-7h2CO2iirjQwjIqiSw9P3EuGjvVoOKpwyjJ1x2_6XbR6Nj26c70gsVrP8KLkQbpVLwuu2yZcyGZlHMzoQw2izaWXTV_lFzB1AQ/file# [following]
--2020-06-17 12:13:36--  https://uce509eadd95d70f30bc44c98d08.dl.dropboxusercontent.com/cd/0/inline/A50F8T3uwdh-7h2CO2iirjQwjIqiSw9P3EuGjvVoOKpwyjJ1x2_6XbR6Nj26c70gsVrP8KLkQbpVLwuu2yZcyGZlHMzoQw2izaWXTV_lFzB1AQ/file
Resolving uce509eadd95d70

In [144]:
!tar -xzvf ru.tar.gz

ru.bin
ru.vec


In [145]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

In [146]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов

def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [147]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [148]:
def items_to_vec(items: np.array) -> np.array:
    return np.asarray([sentence_embedding(item[0])+sentence_embedding(item[1]) for item in items])

Закодируем каждый товар, как сумму векторов полученных применением sentence_embedding к title и description товара

In [149]:
X_train_wv=items_to_vec(X_train)
X_test_wv=items_to_vec(X_test)

In [150]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=100).fit(X_train_wv, y_train)
print('Качество логистической регресии: ',accuracy_score(model.predict(X_test_wv), y_test))

Качество логистической регресии:  0.5523333333333333


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [151]:
from sklearn.svm import LinearSVC
model = LinearSVC(max_iter=70).fit(X_train_wv, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_wv), y_test))

Качество SVC:  0.46266666666666667


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Что ж качество сильно упало по сравнению с предыдущими методами кодирования товаров, я думаю, что это связано с тем, что тексты из интернета не отражают специфику нашей классификации и небольшой размерностью векторов.

In [152]:
from sklearn.preprocessing import normalize
X_train_wv_n=normalize(X_train_wv)
X_test_wv_n=normalize(X_test_wv)

In [153]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=100).fit(X_train_wv_n, y_train)
print('Качество логистической регресии: ',accuracy_score(model.predict(X_test_wv_n), y_test))

Качество логистической регресии:  0.49466666666666664


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [154]:
from sklearn.svm import LinearSVC

model = LinearSVC(max_iter=70).fit(X_train_wv_n, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_wv_n), y_test))

Качество SVC:  0.562


После нормализации качесто на SVC поднялось, а на логистической  регрессии уменьшилось как обычно, но в целом этот метод серьезно проигрывает всем предыдущим.

**Что дальше?**  
Решение каждого пункта 1 балл:


1.   N-Gram модели текстовой классификации
2.   Обучиться на полных данных (контест на kaggle)
3.   Поработать с другими эмбеддингами (word2vec, GloVe).
4.   Использовать Vowpal Wabbit вместо sklearn.
5.   Другие способы токенизации (pymorphy2, spaCy)


Снабжайте код пояснениями и графиками. Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

**N-Gram модели текстовой классификации**

In [155]:
X_train_vec=[item[0]+' '+item[1] for item in X_train]
X_test_vec=[item[0]+' '+item[1] for item in X_test]

In [156]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [157]:
ngram_range=[(1,1),(1,2),(1,3),(1,5),(2,2),(2,5),(3,3)]
svc = LinearSVC(max_iter=70)
tfidf = TfidfVectorizer()
for n in ngram_range:
  tfidf.ngram_range=n
  X_train_t=tfidf.fit_transform(X_train_vec)
  X_test_t=tfidf.transform(X_test_vec)
  svc.fit(X_train_t, y_train)
  print('N-Gram range:', n,'accuracy: ',accuracy_score(svc.predict(X_test_t), y_test))


N-Gram range: (1, 1) accuracy:  0.8352222222222222
N-Gram range: (1, 2) accuracy:  0.8295555555555556
N-Gram range: (1, 3) accuracy:  0.8178888888888889
N-Gram range: (1, 5) accuracy:  0.8033333333333333
N-Gram range: (2, 2) accuracy:  0.7045555555555556
N-Gram range: (2, 5) accuracy:  0.66
N-Gram range: (3, 3) accuracy:  0.5366666666666666


С увелечением n_gram range качество уменьшаеться. Самое высокое качество было достигнуто с n_gram=(1,1)  0.8352222222222222

**Поработать с другими эмбеддингами (word2vec, GloVe).**

Я пыталась по аналогии с заданием wordvec поработать с word2vec. Вектора я тоже брала уже готовые. Насколько у меня получилось судить вам.

In [158]:
from gensim.models.keyedvectors import KeyedVectors
model2 = KeyedVectors.load_word2vec_format('ru.vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [159]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов

def sentence_embedding2(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model2['мама'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model2:
            features += model2[word]
    
    return features

In [160]:
def items_to_vec2(items: np.array) -> np.array:
    return np.asarray([sentence_embedding2(item[0])+sentence_embedding2(item[1]) for item in items])

Закодируем каждый товар, как сумму векторов полученных применением sentence_embedding2 к title и description товара

In [161]:
X_train_wv2=items_to_vec2(X_train)
X_test_wv2=items_to_vec2(X_test)

In [162]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=100).fit(X_train_wv2, y_train)
print('Качество логистической регресии: ',accuracy_score(model.predict(X_test_wv2), y_test))

Качество логистической регресии:  0.6615555555555556


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [163]:
from sklearn.svm import LinearSVC
model = LinearSVC(max_iter=70).fit(X_train_wv2, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_wv2), y_test))

Качество SVC:  0.5385555555555556


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Нормализуем данные

In [164]:
from sklearn.preprocessing import normalize
X_train_wv2_n=normalize(X_train_wv2)
X_test_wv2_n=normalize(X_test_wv2)

In [165]:
model = LogisticRegression(max_iter=100).fit(X_train_wv2_n, y_train)
print('Качество логистической регресии: ',accuracy_score(model.predict(X_test_wv2_n), y_test))

Качество логистической регресии:  0.6141111111111112


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [166]:
model = LinearSVC(max_iter=70).fit(X_train_wv2_n, y_train)
print('Качество SVC: ',accuracy_score(model.predict(X_test_wv2_n), y_test))

Качество SVC:  0.6686666666666666


Что ж этот способ получился получше, чем fasttext в задании wordvec, но качество все еще не очень высокое (0.6686666666666666 максимальное) и проигрывает всем остальным способам векторизации текстов.